In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Utilities as U
from mppi.Utilities import Tools

In [3]:
omp = 1
mpi = 4

# Tutorial for the YamboCalculator class

This tutorial describes the usage of the YamboCalculator class, that manages the run of (many) calculations in
parallel with the Yambo package.

## Perform an Hartree-Fock computation for Silicon

__Follow the tutorial for the QeCalculator to produce the .save folder needed to generate the yambo SAVE.__

The first step needed to perform a Yambo computation is to generate the SAVE folder from a QuantumESPRESSO
computation. 

The mppi.Utilities module has a function that perform this task

In [4]:
source_dir = 'QeCalculator_test/outdir_nscf/bands_8.save'
run_dir = 'YamboCalculator_test'

In [5]:
Tools.build_SAVE(source_dir,run_dir)

Create folder YamboCalculator_test
Executing command: cd QeCalculator_test/outdir_nscf/bands_8.save; p2y
Create a symlink of /home/marco/Applications/MPPI/sphinx_source/tutorials/QeCalculator_test/outdir_nscf/bands_8.save/SAVE in YamboCalculator_test
Executing command: cd YamboCalculator_test;OMP_NUM_THREADS=1 mpirun -np 1 yambo


Now the YamboInput class can create the yambo input object.

We consider a HF computation

In [6]:
exx = 3.0 # Hartree
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
inp.set_kRange(1,2) #restrict the analysis to the first two kpoints
inp['variables']['EXXRLvcs'] = [exx*1e3,'mHa']
name = 'hf_exx'+str(exx)
jobname = 'hf_job_exx'+str(exx)
inp

{'args': 'yambo -x -V rl',
 'folder': 'YamboCalculator_test',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [2133.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [3000.0, 'mHa'],
  'VXCRLvcs': [13107.0, 'RL'],
  'QPkrange': [[1, 2, 1, 8], '']}}

To run the computation we create an istance of YamboCalculator. This object behaves almost exactly as
the QeCalculator for what concerns the user interface.

The first step is to create an istance of the RunRules class that contains the options of the calculator

In [8]:
rr = C.RunRules(mpi=mpi,omp_num_threads=omp)
rr

{'scheduler': 'direct', 'mpi': 4, 'omp_num_threads': 1}

In [9]:
C.YamboCalculator?

Init signature:
C.YamboCalculator(
    runRules,
    executable='yambo',
    skip=True,
    clean_restart=True,
    dry_run=False,
    wait_end_run=True,
    activate_BeeOND=False,
    verbose=True,
    **kwargs,
)
Docstring:     
Perform a Yambo calculation. The parameters used to define the parellelization
strategy are provided in the `runRules` object.

Parameters:
   runRulues (:class:`RunRules`) : instance of the :class:`RunRules` class
   executable (:py:class:`string`) : set the executable (yambo, ypp, yambo_rt, ...) of the Yambo package
   skip (:py:class:`bool`) : if True evaluate if the computation can be skipped. This is done by checking that the
        report file built by yambo exsists and contains the string `game_over`, defined as a data member of this class
   clean_restart (:py:class:`bool`) : if True delete the folder with the output files and the database before running the computation
   dry_run (:py:class:`bool`) : with this option enabled the calculator setup the

In [10]:
code = C.YamboCalculator(rr)
code.global_options()

Initialize a Yambo calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 4,
 'omp_num_threads': 1,
 'executable': 'yambo',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

The structure of the folder in which yambo writes its results is governed by the name and jobname 
variables. It is possible to provide only the name variable. 

The effect of this choice can be seen in the command string executed by the calculator.

In [11]:
result = code.run(run_dir=run_dir,input=inp,name=name,jobname=jobname)
result

run command: mpirun -np 4 yambo -F hf_exx3.0.in -J hf_job_exx3.0 -C hf_exx3.0
computation hf_exx3.0 is running...
computation hf_exx3.0 ended
Run performed in 18s


{'output': ['YamboCalculator_test/hf_exx3.0/o-hf_job_exx3.0.hf'],
 'report': 'YamboCalculator_test/hf_exx3.0/r-hf_job_exx3.0_HF_and_locXC',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_job_exx3.0/ndb.HF_and_locXC'}

In this case yambo create in the ``run_dir`` the hf_exx3.0 folder that contains the o- _output_ files and the hf_job_exx3.0 that contains the .ndb databases.
Result is a dictionary that contain the names of the o- file and the databases created by yambo.

Instead, if we only provide the name parameter all the files are written by yambo in the `name` folder

In [12]:
result = code.run(run_dir=run_dir,input=inp,name=name+'_only')
result

run command: mpirun -np 4 yambo -F hf_exx3.0_only.in -J hf_exx3.0_only -C hf_exx3.0_only
computation hf_exx3.0_only is running...
computation hf_exx3.0_only ended
Run performed in 21s


{'output': ['YamboCalculator_test/hf_exx3.0_only/o-hf_exx3.0_only.hf'],
 'report': 'YamboCalculator_test/hf_exx3.0_only/r-hf_exx3.0_only_HF_and_locXC',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_exx3.0_only/ndb.HF_and_locXC'}

### Usage of the skip parameter

If we repeat a calculation that has been already performed and skip = True the class skip its computation, for instance

In [16]:
result = code.run(run_dir=run_dir,input=inp,name=name,jobname=jobname,skip=True)
result

Skip the run of hf_exx3.0


{'output': ['YamboCalculator_test/hf_exx3.0/o-hf_job_exx3.0.hf'],
 'report': 'YamboCalculator_test/hf_exx3.0/r-hf_job_exx3.0_HF_and_locXC',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_job_exx3.0/ndb.HF_and_locXC'}

Instead, if skip is False the folders with the results are erased and the computation run again

In [17]:
result = code.run(run_dir=run_dir,input=inp,name=name+'_only',skip=False)
result

delete folder: YamboCalculator_test/hf_exx3.0_only
run command: mpirun -np 4 yambo -F hf_exx3.0_only.in -J hf_exx3.0_only -C hf_exx3.0_only
computation hf_exx3.0_only is running...
computation hf_exx3.0_only ended
Run performed in 23s


{'output': ['YamboCalculator_test/hf_exx3.0_only/o-hf_exx3.0_only.hf'],
 'report': 'YamboCalculator_test/hf_exx3.0_only/r-hf_exx3.0_only_HF_and_locXC',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_exx3.0_only/ndb.HF_and_locXC'}

The clean of the results folder can be suppressed with the option clean_restart=False

In [15]:
result = code.run(run_dir=run_dir,input=inp,name=name+'_only',skip=False,clean_restart=False)
result

run performed starting from existing results
run command: mpirun -np 4 yambo -F hf_exx3.0_only.in -J hf_exx3.0_only -C hf_exx3.0_only
computation hf_exx3.0_only is running...
computation hf_exx3.0_only ended


{'output': ['YamboCalculator_test/hf_exx3.0_only/o-hf_exx3.0_only.hf',
  'YamboCalculator_test/hf_exx3.0_only/o-hf_exx3.0_only.hf_01',
  'YamboCalculator_test/hf_exx3.0_only/o-hf_exx3.0_only.hf_03',
  'YamboCalculator_test/hf_exx3.0_only/o-hf_exx3.0_only.hf_02'],
 'report': 'YamboCalculator_test/hf_exx3.0_only/r-hf_exx3.0_only_HF_and_locXC_02',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_exx3.0_only/ndb.HF_and_locXC'}

__In this case there can be more replica of the report and of the output files, so care has to taken to ensure that
results contains the right elements__

### Test of the slurm scheduler

If the `slurm` scheduler is chosen the calculator prepare the slurm script and submit it. 

In this case the mpi variable is not used and the calculator is set using the `ntasks_per_node`,`nodes`
and `cpus_per_task variables`, apart from the `omp_num_threads`.

In [18]:
rr = C.RunRules(scheduler='slurm',ntasks_per_node=4,omp_num_threads=2)
rr

{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 4,
 'cpus_per_task': 1,
 'omp_num_threads': 2,
 'gpus_per_node': None,
 'memory': '124GB',
 'time': None,
 'partition': None,
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None}

In [19]:
code = C.YamboCalculator(rr)
code.global_options()

Initialize a Yambo calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 4,
 'cpus_per_task': 1,
 'omp_num_threads': 2,
 'gpus_per_node': None,
 'memory': '124GB',
 'time': None,
 'partition': None,
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'executable': 'yambo',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [21]:
results = code.run(run_dir=run_dir,input=inp,name=name,jobname=jobname,dry_run=True,skip=False,clean_restart=False)
results

run performed starting from existing results
run command: mpirun -np 4 yambo -F hf_exx3.0.in -J hf_job_exx3.0 -C hf_exx3.0
Dry_run option active. Script not submitted
The wait_end_run is False or the dry_run option is active. The calculator proceedes to the postprocessing
Run performed in 18s


{'output': ['YamboCalculator_test/hf_exx3.0/o-hf_job_exx3.0.hf'],
 'report': 'YamboCalculator_test/hf_exx3.0/r-hf_job_exx3.0_HF_and_locXC',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'HF_and_locXC': 'YamboCalculator_test/hf_job_exx3.0/ndb.HF_and_locXC'}

The slurm script is written in the run_dir. The execution of the run requires that the slurm scheduler is installed.

## Perform a GW computation for Silicon

We make usage of the YamboCalculator to perform a different yambo computation. In this way we control
how this class manage the output files and the ndb database in various cases.

In [22]:
rr = C.RunRules(mpi=mpi,omp_num_threads=omp)
code = C.YamboCalculator(rr)

Initialize a Yambo calculator with scheduler direct


In [23]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp.set_kRange(1,2)
#inp

In [24]:
result = code.run(input=inp,run_dir=run_dir,name='qp_test1')
result

run command: mpirun -np 4 yambo -F qp_test1.in -J qp_test1 -C qp_test1
computation qp_test1 is running...
computation qp_test1 ended
Run performed in 32s


{'output': ['YamboCalculator_test/qp_test1/o-qp_test1.qp'],
 'report': 'YamboCalculator_test/qp_test1/r-qp_test1_HF_and_locXC_gw0_dyson_em1d_ppa',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'pp': 'YamboCalculator_test/qp_test1/ndb.pp',
 'QP': 'YamboCalculator_test/qp_test1/ndb.QP',
 'HF_and_locXC': 'YamboCalculator_test/qp_test1/ndb.HF_and_locXC',
 'dipoles': 'YamboCalculator_test/qp_test1/ndb.dipoles'}

Perform the same computation but specify also a jobname 

In [25]:
result = code.run(input = inp, run_dir = run_dir, name='qp_test2', jobname = 'qp_job_test2')
result

run command: mpirun -np 4 yambo -F qp_test2.in -J qp_job_test2 -C qp_test2
computation qp_test2 is running...
computation qp_test2 ended
Run performed in 38s


{'output': ['YamboCalculator_test/qp_test2/o-qp_job_test2.qp'],
 'report': 'YamboCalculator_test/qp_test2/r-qp_job_test2_HF_and_locXC_gw0_dyson_em1d_ppa',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'pp': 'YamboCalculator_test/qp_job_test2/ndb.pp',
 'QP': 'YamboCalculator_test/qp_job_test2/ndb.QP',
 'HF_and_locXC': 'YamboCalculator_test/qp_job_test2/ndb.HF_and_locXC',
 'dipoles': 'YamboCalculator_test/qp_job_test2/ndb.dipoles'}

## Test of the ExtendOut option in the input file

The ExtendOut option enables the writing of all the variables in the output files of Yambo. 
This feature has no effect for an HF computation and we test for a QP one:

In [26]:
inp = I.YamboInput(args='yambo -d -k hartee -g n -p p -V qp',folder=run_dir)
inp.set_kRange(1,2)
inp.set_extendOut()
#inp

In [27]:
result = code.run(input = inp, run_dir = run_dir,name='qp_test_ExtendOut')
result

run command: mpirun -np 4 yambo -F qp_test_ExtendOut.in -J qp_test_ExtendOut -C qp_test_ExtendOut
computation qp_test_ExtendOut is running...
computation qp_test_ExtendOut ended
Run performed in 31s


{'output': ['YamboCalculator_test/qp_test_ExtendOut/o-qp_test_ExtendOut.qp'],
 'report': 'YamboCalculator_test/qp_test_ExtendOut/r-qp_test_ExtendOut_HF_and_locXC_gw0_dyson_em1d_ppa',
 'dft': 'YamboCalculator_test/SAVE/ns.db1',
 'pp': 'YamboCalculator_test/qp_test_ExtendOut/ndb.pp',
 'QP': 'YamboCalculator_test/qp_test_ExtendOut/ndb.QP',
 'HF_and_locXC': 'YamboCalculator_test/qp_test_ExtendOut/ndb.HF_and_locXC',
 'dipoles': 'YamboCalculator_test/qp_test_ExtendOut/ndb.dipoles'}

you can check that the o- files contain more information. This feature is managed by the YamboParser class of the package.

## Perform a ypp computation

The YamboCalculator class can manage also ypp computation.

Let's see an example by performing a band calculation along a path

In [28]:
inp = I.YamboInput(args='ypp -s b',folder=run_dir,filename='ypp.in') 
inp

{'args': 'ypp -s b',
 'folder': 'YamboCalculator_test',
 'filename': 'ypp.in',
 'arguments': [],
 'variables': {'INTERP_Shell_Fac': [20.0, ''],
  'INTERP_NofNN': [1.0, ''],
  'OutputAlat': [0.0, ''],
  'BANDS_steps': [10.0, ''],
  'PROJECT_mode': 'none',
  'INTERP_mode': 'NN',
  'cooIn': 'rlu',
  'cooOut': 'rlu',
  'CIRCUIT_E_DB_path': 'none',
  'BANDS_bands': [[1, 8], '']}}

We define a calculator for ypp. This calculation requires 1 mpirun (see yambo for further information)

In [30]:
rr['mpi']=1
code = C.YamboCalculator(rr,executable='ypp')
code.global_options()

Initialize a Yambo calculator with scheduler direct


{'scheduler': 'direct',
 'mpi': 1,
 'omp_num_threads': 1,
 'executable': 'ypp',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

Set the input parameter to perform the band computation along a path

In [31]:
# in alat
G = [0.,0.,0.]
X = [1.,0.,0.]
L = [0.5,0.5,0.5]
K = [1.0,0.5,0.]
path = [L,G,X,K,G]

band_range = [2,5]
bands_step = 30

In [32]:
# scissor
# inp['variables']['GfnQP_E'] = [1.0,1.0,1.0]

# band structure
# Some methods that perform these operation can be added in the YamboInput class
inp['variables']['BANDS_steps'] = [bands_step,'']
inp['variables']['BANDS_bands'] = [band_range,'']
inp['variables']['BANDS_kpts'] = [path,'']
inp['variables']['cooIn'] = 'alat'
inp['variables']['cooOut'] = 'alat'
inp

{'args': 'ypp -s b',
 'folder': 'YamboCalculator_test',
 'filename': 'ypp.in',
 'arguments': [],
 'variables': {'INTERP_Shell_Fac': [20.0, ''],
  'INTERP_NofNN': [1.0, ''],
  'OutputAlat': [0.0, ''],
  'BANDS_steps': [30, ''],
  'PROJECT_mode': 'none',
  'INTERP_mode': 'NN',
  'cooIn': 'alat',
  'cooOut': 'alat',
  'CIRCUIT_E_DB_path': 'none',
  'BANDS_bands': [[2, 5], ''],
  'BANDS_kpts': [[[0.5, 0.5, 0.5],
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [1.0, 0.5, 0.0],
    [0.0, 0.0, 0.0]],
   '']}}

Also for these kind of computation we can use the skip and the clean_restart options

In [33]:
result = code.run(run_dir=run_dir,input=inp,name='bands_test1',skip=False,clean_restart=False)
result

run performed starting from existing results
run command: mpirun -np 1 ypp -F bands_test1.in -J bands_test1 -C bands_test1
computation bands_test1 is running...
computation bands_test1 ended


{'output': ['YamboCalculator_test/bands_test1/o-bands_test1.bands_interpolated'],
 'report': 'YamboCalculator_test/bands_test1/r-bands_test1_electrons_bnds',
 'dft': 'YamboCalculator_test/SAVE/ns.db1'}

In this case the report does not contains the `time_profile` string so the simulation time is not provided. 

Result can be parsed using the YamboParser class of this package.